In [7]:
%load_ext autoreload

%autoreload 2

from __future__ import division, print_function

import cPickle as pickle
from glob import glob
import subprocess
import sys
import string
import os
import random
import numpy as np
from os.path import join

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
name = "lband"

In [55]:
d = join("derivative-runs",name)
params = pickle.load(open(join(d,"params.pickle"),"rb"))
F = threebody.Fitter(**params)
try:
    F.best_parameters = pickle.load(
        open(join(d,"best_parameters.pickle"),"rb"))
except IOError:
    pass
derivatives = {}
for p in F.parameters:
    derivatives[p] = np.load(join(d,"%s.npy" % p))
print("Goodness of fit:",F.goodness_of_fit())
M, linear_parameters = F.compute_linear_matrix()
lv, linear_parameters = F.compute_linear_parts()
for i, p in enumerate(linear_parameters):
    derivatives[p] = M[:,i]

ps = F.parameters + linear_parameters
Abig = np.array([derivatives[p] 
                 for p in ps], 
                dtype=np.longdouble).T
#print(Abig.shape)
result = threebody.lstsq_with_errors(Abig, 
                                     F.residuals(), 
                                     F.phase_uncerts)

for i,p in enumerate(ps):
    if p in F.parameters:
        v = F.best_parameters[p]
    elif p in linear_parameters:
        v = lv[linear_parameters.index(p)]
    else:
        v = 0
    print(p, 
          threebody.fmt(v-result.x[i], result.uncerts[i]), 
          result.uncerts[i])

Goodness of fit: 35964.2426404
asini_i 1.2175257(3)e0 2.63787630239e-07
pb_i 1.6293946(7)e0 6.78439451839e-07
eps1_i 6.879(2)e-4 2.05113394164e-07
eps2_i -9.1375(18)e-5 1.81444874624e-08
tasc_i 4.0771674(7)e-1 6.53765097781e-08
acosi_i 1.48968(19)e0 0.000190225026422
q_i 1.37415(5)e-1 4.58682128044e-06
asini_o 7.4672659(13)e1 1.27770208215e-05
pb_o 3.2725712(11)e2 0.000111175180261
eps1_o 3.518607(5)e-2 4.87421507821e-08
eps2_o -3.46273(16)e-3 1.62125743842e-07
tasc_o 3.1393539(11)e2 0.000111119145954
acosi_o 9.1371(12)e1 0.0124057618027
delta_lan 5.6(9)e-6 8.51042143298e-07
delta -6.8(19)e-7 1.89080335228e-07
dgamma 1(3)e-2 0.0277406178145
dbeta 5(5)e-2 0.053267632291
DM_0000 1.5(12)e2 116.254445136
DM_0001 1.5(12)e2 116.25438626
DM_0002 1.5(12)e2 116.253921211
DM_0003 1.5(12)e2 116.254014236
DM_0004 1.5(12)e2 116.253785841
DM_0005 1.5(12)e2 116.254079955
DM_0006 1.5(12)e2 116.253838312
DM_0007 1.5(12)e2 116.25419967
DM_0008 1.5(12)e2 116.254118995
DM_0009 1.5(12)e2 116.255546564
DM_0

In [56]:
# FIXME
bp = F.best_parameters.copy()
for p in F.parameters:
    i = ps.index(p)
    bp[p] -= result.x[i]
pre = F.goodness_of_fit(F.best_parameters)
print("pre-improvement:", pre)
post = F.goodness_of_fit(bp)
print("post-improvement:", post)
if post<pre:
    with open("derivative-best-parameters.pickle","wb") as f:
        pickle.dump(bp, f)

pre-improvement: 35964.2426404
post-improvement: 319961.550896


In [57]:
F.parameters

['asini_i',
 'pb_i',
 'eps1_i',
 'eps2_i',
 'tasc_i',
 'acosi_i',
 'q_i',
 'asini_o',
 'pb_o',
 'eps1_o',
 'eps2_o',
 'tasc_o',
 'acosi_o',
 'delta_lan',
 'delta',
 'dgamma',
 'dbeta']

In [58]:
np.median(F.phase_uncerts)/F.reference_f0*1e6

1.24043120876